In [1]:
import os
import hmac
import hashlib
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

# Architecture Class

## IoTDevice

The `IoTDevice` class is designed to establish a secure authentication and communication mechanism using a shared vault and AES encryption. Below is a breakdown of its functionality:  

### **1. Initialization (`__init__`)**  
- Each device has a unique `device_id`.  
- It maintains a **vault** (a shared secret key storage).  
- Several attributes (`C2`, `r2`, `t1`, `session_key`, `trasmitted_data`) track authentication state.  
- A debug mode (`debug=True`) enables logging for troubleshooting.  

### **2. Connection Request (`create_connection_request`)**  
- Generates an authentication request containing the `device_id` and `session_id`.  

### **3. Authentication Mechanism**  

#### **Challenge-Response (`generate_response`)**  
1. Computes a secret key `k1` by XOR-ing values from the vault.  
2. Generates `t1` (a random nonce).  
3. Concatenates `r1` (server challenge) and `t1`.  
4. Generates a new challenge (`C2`, `r2`) for the server.  
5. Encrypts the concatenated data using **AES-CBC** mode with `k1`.  
6. Returns the encrypted response with an IV.  

#### **Challenge Generation (`generate_challenge`)**  
- Selects `p=3` random indices from the vault.  
- Generates an 8-byte random challenge (`r1`).  

#### **Response Verification (`verify_response`)**  
1. Computes `k2` (similar to `k1`).  
2. Uses `k2` to decrypt the received response.  
3. Validates `r2` (ensuring server authenticity).  
4. Computes the **session key** as `t1 XOR t2`.  

### **4. Secure Communication**  

#### **Encrypt Message (`encrtpt_msg`)**  
- Encrypts a message using **AES-CBC** with the session key.  
- Prepends the IV for decryption.  

#### **Decrypt Message (`decrypt_msg`)**  
- Extracts IV and decrypts the message using the session key.  

### **5. Vault Update (`update_vault`)**  
- Computes an **HMAC-based key** to modify the vault securely.  
- Uses XOR to update vault entries, ensuring forward secrecy.

In [9]:
class IoTDevice:
    def __init__(self, device_id, shared_vault, debug=False):
        self.device_id = device_id
        self.vault = shared_vault.copy()
        self.C2 = None
        self.r2 = None
        self.t1 = None
        self.session_key = None
        self.trasmitted_data = None
        self.debug = debug

    def _debug(self, *args):
        if self.debug:
            print(*args)

    def create_connection_request(self, session_id):
        self._debug(f"IoTDevice: Creating connection request with session_id: {session_id} and device_id: {self.device_id}")
        return {"device_id": self.device_id, "session_id": session_id}

    def generate_response(self, challenge_indices, r1):
        self._debug(f"IoTDevice: Generating response with challenge_indices: {challenge_indices} and r1: {r1}")
        # Step 1: Compute k1 as XOR of keys in the indices
        k1 = self.vault[challenge_indices[0]]
        for i in challenge_indices[1:]:
            k1 = bytes(a ^ b for a, b in zip(k1, self.vault[i]))
        self._debug("IoTDevice: K1 (XOR Vault): ", k1)
        self._debug("IoTDevice: Length of K1: ", len(k1))
        if len(k1) not in (16, 24, 32):
            raise ValueError("IoTDevice: K1 must be 16, 24, or 32 bytes long for AES encryption")
        # Step 2: Generate t1 (random number for session key generation)
        t1 = os.urandom(len(r1))
        self.t1 = t1
        self._debug("IoTDevice: R1 from server: ", r1)
        self._debug("IoTDevice: T1: ", t1)
        self._debug("IoTDevice: Length of T1: ", len(t1))

        # Step 3: Concatenate r1 and t1
        concat_r1_t1 = r1 + t1
        self._debug("IoTDevice: Concatenated R1 and T1: ", concat_r1_t1)

        # Step 5: Generate C2 (challenge for server)
        c2, r2 = self.generate_challenge()
        self.C2 = c2
        self.r2 = r2
        self._debug("IoTDevice: Generated C2: ", c2)
        self._debug(f"IoTDevice: Generated R2: {r2}")

        # Step 6: Concatenate response and c2 and r2
        data_to_encrypt = concat_r1_t1 + bytes(c2) + bytes(r2)
        self._debug("IoTDevice: Data to Encrypt: ", data_to_encrypt)
        cipher = AES.new(k1, AES.MODE_CBC)  # Use CBC mode for encryption
        iv = cipher.iv  # Initialization vector for CBC
        encrypted_data = cipher.encrypt(pad(data_to_encrypt, AES.block_size))
        self._debug("IoTDevice: Encrypted Data: ", encrypted_data)

        # Include the IV with the encrypted data for decryption
        final_response = iv + encrypted_data
        self._debug("IoTDevice: Final Response (IV + Encrypted Data): ", final_response)
        return {"response": final_response}

    def generate_challenge(self, p=3):
        indices = list(set(os.urandom(1)[0] % len(self.vault) for _ in range(p)))
        while len(indices) < p:
            indices.append(os.urandom(1)[0] % len(self.vault))  # Ensure distinct indices
        r1 = os.urandom(8)  # Random number
        self._debug(f"IoTDevice: Client selected indices: {indices}")
        self._debug(f"IoTDevice: Random number r1: {r1}")
        return indices, r1

    def verify_response(self, response):
        # Compute k2 from vault based on C2 challenge indices
        k2 = self.vault[self.C2[0]]
        for i in self.C2[1:]:
            k2 = bytes(a ^ b for a, b in zip(k2, self.vault[i]))
        self._debug("IoTDevice: K2 Partial (XOR Vault): ", k2)
        # Step 1: k2 XOR t1
        k2 = bytes(a ^ b for a, b in zip(k2, self.t1))
        self._debug(f"IoTDevice: K2 (XOR Vault XOR T1): {k2}, Length: {len(k2)}")
        if len(k2) < 16:
            k2 = k2.ljust(16, b'\0')  # Pad with zeroes
        elif len(k2) > 32:
            k2 = k2[:32]  # Truncate to 32 bytes
        elif len(k2) not in (16, 24, 32):
            k2 = k2.ljust(24, b'\0')  # Default to 24 bytes
        self._debug(f"IoTDevice: K2 (after padding): {k2}, Length: {len(k2)}")
        # Step 2: Decrypt the response
        iv = response[:AES.block_size]
        cipher = AES.new(k2, AES.MODE_CBC, iv=iv)
        decrypted_data = unpad(cipher.decrypt(response[AES.block_size:]), AES.block_size)
        self._debug("IoTDevice: Decrypted Data: ", decrypted_data)
        r2 = decrypted_data[:8]
        self._debug("IoTDevice: R2 from server: ", r2)
        if r2 != bytes(self.r2):
            raise ValueError("IoTDevice: R2 does not match")
        self._debug("IoTDevice: R2 matches")
        
        # Step 3: Session key is t1 XOR t2 (t2 is the remaining bytes)
        t2 = decrypted_data[8:]
        self._debug("IoTDevice: T2 from server: ", t2)
        self._debug("IoTDevice: T1: ", self.t1)
        session_key = bytes(a ^ b for a, b in zip(self.t1, t2))
        if len(session_key) < 16:
            session_key = session_key.ljust(16, b'\0')
        self._debug("IoTDevice: Session Key: ", session_key)
        self.session_key = session_key

    def encrtpt_msg(self, msg):
        self.trasmitted_data = msg
        cipher = AES.new(self.session_key, AES.MODE_CBC)
        iv = cipher.iv
        encrypted_data = cipher.encrypt(pad(msg, AES.block_size))
        return iv + encrypted_data

    def decrypt_msg(self, msg):
        iv = msg[:AES.block_size]
        cipher = AES.new(self.session_key, AES.MODE_CBC, iv=iv)
        decrypted_data = unpad(cipher.decrypt(msg[AES.block_size:]), AES.block_size)
        self.trasmitted_data = decrypted_data
        return decrypted_data

    def update_vault(self):
        # Digest all the secure vault data
        data_to_digest = b''.join(self.vault)
        h_key = hmac.new(self.trasmitted_data, data_to_digest, hashlib.sha256).digest()
        self._debug(f"IoTDevice: Generated HMAC key: {h_key}")
        j = 10
        k = 16
        data_parts = [data_to_digest[i:i+k] for i in range(0, len(data_to_digest), k)]
        self._debug(f"IoTDevice: Data Parts: {data_parts}")
        # For each part XOR with the HMAC key and update the vault
        for i, part in enumerate(data_parts):
            self.vault[i] = bytes(a ^ b for a, b in zip(part, h_key))
        self._debug(f"IoTDevice: Updated Vault: {self.vault}")

## IoTServer

The `IoTServer` class is responsible for handling authentication requests, verifying responses, and securing communication with IoT devices using AES encryption and a shared vault. Below is a breakdown of its functionality:  

### **1. Initialization (`__init__`)**  
- Stores a **shared vault** containing secret keys.  
- Maintains a list of **valid device IDs** for authentication.  
- Several attributes (`C1`, `C2`, `r1`, `r2`, `session_key`, `trasmitted_data`) track authentication state.  
- A debug mode (`debug=True`) enables logging for troubleshooting.  

### **2. Connection Handling (`handle_connection_request`)**  
- Checks if the device ID is in the list of **valid devices**.  
- Accepts or rejects the connection request.  

### **3. Authentication Mechanism**  

#### **Challenge Generation (`generate_challenge`)**  
1. Selects `p=3` random indices from the vault.  
2. Generates an **8-byte random number (`r1`)** as a challenge.  

#### **Response Verification (`verify_response`)**  
1. **Computes `k1`** by XOR-ing values from the vault (using indices from `C1`).  
2. **Decrypts the response** using `k1` (AES-CBC mode).  
3. Extracts:
   - `r1` (validates IoT device challenge response).  
   - `t1` (random number from the IoT device).  
   - `C2` (new challenge for the IoT device).  
   - `r2` (random value for verification).  
4. If `r1` doesn't match, authentication fails.  
5. Computes `k2` based on `C2` (XOR of selected vault indices).  
6. Generates `t2` (random nonce for session key derivation).  
7. Encrypts `r2 + t2` using `k2` (AES-CBC).  
8. **Derives the session key** as `t1 XOR t2`.  

### **4. Secure Communication**  

#### **Encrypt Message (`encrtpt_msg`)**  
- Encrypts a message using **AES-CBC** with the session key.  
- Prepends the IV for decryption.  

#### **Decrypt Message (`decrypt_msg`)**  
- Extracts IV and decrypts the message using the session key.  

### **5. Vault Update (`update_vault`)**  
- Computes an **HMAC-based key** to securely modify the vault.  
- Uses XOR to update vault entries, ensuring forward secrecy.  


In [10]:
class IoTServer:
    def __init__(self, shared_vault, valid_device_ids, debug=False):
        self.vault = shared_vault.copy()
        self.valid_device_ids = valid_device_ids
        self.C1 = None
        self.C2 = None
        self.r1 = None
        self.r2 = None
        self.session_key = None
        self.trasmitted_data = None
        self.debug = debug

    def _debug(self, *args):
        if self.debug:
            print(*args)

    def handle_connection_request(self, request):
        device_id = request.get("device_id")
        session_id = request.get("session_id")
        if device_id in self.valid_device_ids:
            self._debug(f"IoTServer: Connection request accepted for Device ID: {device_id}, Session ID: {session_id}")
            return True
        else:
            self._debug(f"IoTServer: Connection request denied for Device ID: {device_id}")
            return False

    def generate_challenge(self, p=3):
        indices = list(set(os.urandom(1)[0] % len(self.vault) for _ in range(p)))
        while len(indices) < p:
            indices.append(os.urandom(1)[0] % len(self.vault))  # Ensure distinct indices
        r1 = os.urandom(8)  # Random number
        self._debug(f"IoTServer: Server selected indices: {indices}")
        self._debug(f"IoTServer: Random number r1: {r1}")
        return indices, r1

    def verify_response(self, response):
        # Step 1: Compute k1 as XOR of keys in the indices (from C1)
        k1 = self.vault[self.C1[0]]
        for i in self.C1[1:]:
            k1 = bytes(a ^ b for a, b in zip(k1, self.vault[i]))
        self._debug("IoTServer: K1 Generated from Local Challenge (XOR Vault): ", k1)
        # Decrypt the response using k1
        iv = response[:AES.block_size]  # Extract IV (16 bytes for AES)
        encrypted_data = response[AES.block_size:]  # Remaining is the encrypted data
        cipher = AES.new(k1, AES.MODE_CBC, iv)
        decrypted_data = unpad(cipher.decrypt(encrypted_data), AES.block_size)
        self._debug("IoTServer: Decrypted Data: ", decrypted_data)
        # Extract r1, t1, c2, and r2 from decrypted data
        r1 = decrypted_data[:8]
        t1 = decrypted_data[8:16]
        c2 = decrypted_data[16:19]
        r2 = decrypted_data[19:]
        self._debug("IoTServer: Extracted R1: ", r1)
        self._debug("IoTServer: Extracted T1: ", t1)
        self._debug("IoTServer: Extracted C2: ", c2)
        self._debug("IoTServer: Extracted R2: ", r2)
        if r1 != self.r1:
            self._debug("IoTServer: R1 does not match")
            return False
        else:
            self._debug("IoTServer: R1 matches")
            
        # Step 2: Compute k2 based on C2
        c2 = list(c2)
        self._debug("IoTServer: C2: ", c2)
        k2 = self.vault[c2[0]]
        for i in c2[1:]:
            k2 = bytes(a ^ b for a, b in zip(k2, self.vault[i]))
        self._debug("IoTServer: K2 (XOR Vault for C2): ", k2)
        # Step 3: Generate t2 (random number for session key generation)
        t2 = os.urandom(len(t1))
        self._debug("IoTServer: T2: ", t2)
        # Step 4: XOR k2 with t1 to generate key for decrypting server's message
        k2 = bytes(a ^ b for a, b in zip(k2, t1))
        # Adjust K2 length to be valid for AES
        if len(k2) < 16:
            k2 = k2.ljust(16, b'\0')
        elif len(k2) > 32:
            k2 = k2[:32]
        elif len(k2) not in (16, 24, 32):
            k2 = k2.ljust(24, b'\0')
        data_to_encrypt = r2 + t2
        self._debug("IoTServer: Data to Encrypt: ", data_to_encrypt)
        cipher = AES.new(k2, AES.MODE_CBC)
        iv = cipher.iv
        encrypted_data = cipher.encrypt(pad(data_to_encrypt, AES.block_size))
        self._debug("IoTServer: Encrypted Data: ", encrypted_data)
        self._debug(f"IoTServer: T1 for session key: {t1}")
        self._debug(f"IoTServer: T2 for session key: {t2}")
        session_key = bytes(a ^ b for a, b in zip(t1, t2))
        if len(session_key) < 16:
            session_key = session_key.ljust(16, b'\0')
        self.session_key = session_key
        self._debug("IoTServer: Session Key: ", session_key)
        
        return iv + encrypted_data

    def encrtpt_msg(self, msg):
        self.trasmitted_data = msg
        cipher = AES.new(self.session_key, AES.MODE_CBC)
        iv = cipher.iv
        encrypted_data = cipher.encrypt(pad(msg, AES.block_size))
        return iv + encrypted_data

    def decrypt_msg(self, msg):
        iv = msg[:AES.block_size]
        cipher = AES.new(self.session_key, AES.MODE_CBC, iv=iv)
        decrypted_data = unpad(cipher.decrypt(msg[AES.block_size:]), AES.block_size)
        self.trasmitted_data = decrypted_data
        return decrypted_data

    def update_vault(self):
        # Digest all the secure vault data
        data_to_digest = b''.join(self.vault)
        h_key = hmac.new(self.trasmitted_data, data_to_digest, hashlib.sha256).digest()
        self._debug(f"IoTServer: New HMAC Key: {h_key}")
        j = 10
        k = 16
        data_parts = [data_to_digest[i:i+k] for i in range(0, len(data_to_digest), k)]
        self._debug(f"IoTServer: Data Parts: {data_parts}")
        # For each part XOR with the HMAC key and update the vault
        for i, part in enumerate(data_parts):
            self.vault[i] = bytes(a ^ b for a, b in zip(part, h_key))
        self._debug(f"IoTServer: Updated Vault: {self.vault}")

## IoTConnectionManager

The `IoTConnectionManager` class facilitates **secure communication** between an IoT device and a server by handling authentication, message exchange, and session termination.  

### **1. Initialization (`__init__`)**  
- Associates an **IoT device** and an **IoT server** with a unique `session_id`.  
- Maintains a flag (`connection_established`) to track the session status.  

### **2. Connection Establishment (`establish_connection`)**  
1. The device **sends a connection request** containing its `session_id`.  
2. The server **verifies the device ID** and responds with a **challenge (`C1`, `r1`)**.  
3. The device computes a **response** using vault-based encryption and sends it back.  
4. The server **verifies the response** and, if successful, sends back a secure confirmation.  
5. The device validates the server’s response, and **a secure session is established**.  

### **3. Secure Communication**  

#### **Device-to-Server (`send_from_device`)**  
- The device encrypts the message using the **session key** and sends it.  
- The server decrypts and retrieves the original message.  

#### **Server-to-Device (`send_from_server`)**  
- The server encrypts a message using the **session key** and transmits it.  
- The device decrypts and retrieves the original message.  

### **4. Closing the Connection (`close_connection`)**  
- Updates the **vaults** on both the device and server to maintain security.  
- Marks the session as closed to prevent further communication.  


In [38]:
class IoTConnectionManager:
    def __init__(self, device, server, session_id):
        self.device = device
        self.server = server
        self.session_id = session_id
        self.connection_established = False

    def establish_connection(self):
        print("\n=== Starting Connection Establishment ===")
        connection_request = self.device.create_connection_request(self.session_id)
        if not self.server.handle_connection_request(connection_request):
            print("IoTConnectionManager: Connection Denied by Server.")
            return False
        
        challenge_indices, r1 = self.server.generate_challenge()
        self.server.r1 = r1
        self.server.C1 = challenge_indices

        response_packet = self.device.generate_response(challenge_indices, r1)
        device_response = response_packet["response"]

        server_response = self.server.verify_response(device_response)
        if not server_response:
            print("IoTConnectionManager: Server verification failed.")
            return False

        self.device.verify_response(server_response)
        self.connection_established = True
        print("=== Connection Established Successfully ===")
        return True

    def send_from_device(self, message):
        if not self.connection_established:
            print("IoTConnectionManager: Cannot send message. No active connection.")
            return None
        print("\n=== Device to Server Communication ===")
        enc_msg = self.device.encrtpt_msg(message)
        print("Device sent encrypted message: ", enc_msg)
        dec_msg = self.server.decrypt_msg(enc_msg)
        print("Server decrypted message: ", dec_msg)
        return dec_msg

    def send_from_server(self, message):
        if not self.connection_established:
            print("IoTConnectionManager: Cannot send message. No active connection.")
            return None
        print("\n=== Server to Device Communication ===")
        enc_msg = self.server.encrtpt_msg(message)
        print("Server sent encrypted message: ", enc_msg)
        dec_msg = self.device.decrypt_msg(enc_msg)
        print("Device decrypted message: ", dec_msg)
        return dec_msg

    def close_connection(self):
        if not self.connection_established:
            print("IoTConnectionManager: No active connection to close.")
            return
        print("\n=== Closing Connection ===")
        self.device.update_vault()
        self.server.update_vault()
        self.connection_established = False
        print("IoTConnectionManager: Connection closed and vaults updated.")

# Auth Test
## Setup

This code sets up the initial environment for an IoT system where a device and a server can establish a secure connection.

1. **Vault Setup:**
   - `VAULT_SIZE` is set to `10`, meaning there will be 10 random keys generated for the shared vault.
   - `KEY_SIZE` is set to `16`, so each key will be 16 bytes long.
   - `shared_vault` is populated with 10 random 16-byte keys using `os.urandom(KEY_SIZE)`.
   - Each key in the shared vault is printed with an index for reference.

2. **Device and Server Creation:**
   - A `valid_device_ids` set is defined, containing two valid device IDs: `"Device_1"` and `"Device_2"`.
   - The `IoTDevice` class is instantiated with the device ID `"Device_1"` and the shared vault.
   - The `IoTServer` class is instantiated with the shared vault and the set of valid device IDs.

3. **Connection Manager Initialization:**
   - A `session_id` is defined as `"session_123"`.
   - An instance of the `IoTConnectionManager` class is created, which takes the `device`, `server`, and `session_id` as parameters.
   
This setup prepares the environment for the device and server to engage in secure communication through the connection manager.


In [39]:
VAULT_SIZE = 10
KEY_SIZE = 16
shared_vault = [os.urandom(KEY_SIZE) for _ in range(VAULT_SIZE)]
print("Shared Vault: ")
for i, key in enumerate(shared_vault):
    print(f"Index {i}: {key}")
valid_device_ids = {"Device_1", "Device_2"}  # Set of valid device IDs

# Create device and server with the shared vault
device = IoTDevice(device_id="Device_1", shared_vault=shared_vault)
server = IoTServer(shared_vault=shared_vault, valid_device_ids=valid_device_ids)

# Create the connection manager with a session ID
session_id = "session_123"
connection_manager = IoTConnectionManager(device, server, session_id)

Shared Vault: 
Index 0: b'(\xc5\xeb\xc5\xcaq\xc2Yw\x18\x9d\xa2\x98\x97\xff\x80'
Index 1: b'\x85\x8aK\xd6\xba\xcd,\xb5\xec1\x02\x0e\xe0\xd3\xdfu'
Index 2: b'\xbfN\x10\xbd\xbe\x93\x7f\x9f\xd1\x89\\8t,\xf3>'
Index 3: b'-\\\xc3\xb3\x8e\xe6\xdc\xa0\x1d\x03\x94\x83\xdb7OM'
Index 4: b'A\x82\xc5Y\xa1\xb0\xed\\\x1b\xfc\xc7\x1bg\xb9\xa7/'
Index 5: b"\xc2?C'\x1f\xee>nfF\xe9\xa4\xac\x05)D"
Index 6: b'8\xd3\xa6\x8a\x0b4:\xea\xae\x84Axg\x19\xdb\x1d'
Index 7: b'c&\xc3\xb3\xfdx_\xad\xa2Qy\xe5\xe8\x7f\r\xf5'
Index 8: b'\xa58\xc6\xd0#tI\xb5\xb7^C\xc3\xbd\xc5|_'
Index 9: b'b!\xa8\xa5\x94z\x12\xc7\xe9\x1c\xed\x054\xd0\n\xec'


This code cell demonstrates the process of establishing a secure connection between the IoT device and server, followed by bidirectional communication and closing the connection.

1. **Connection Establishment:**
   - The `establish_connection` method of the `IoTConnectionManager` is called to initiate the connection between the device and server.
   - If the connection is successful, the flow continues. Otherwise, a message indicating the failure is printed.

2. **Bidirectional Communication:**
   - **Device to Server Communication:**
     - The `send_from_device` method is called with the message `"Hello from IoT Device"`. This encrypts the message on the device side, sends it to the server, and prints the encrypted message along with the server’s decrypted response.
   - **Server to Device Communication:**
     - The `send_from_server` method is called with the message `"Hello from IoT Server"`. This encrypts the message on the server side, sends it to the device, and prints the encrypted message along with the device’s decrypted response.

3. **Closing the Connection:**
   - The `close_connection` method is called, which updates the vaults of both the device and server (through the `update_vault` method). The connection is then closed.

This code demonstrates the full lifecycle of establishing a connection, communicating securely between the device and server, and properly closing the connection while ensuring that vaults are updated.

In [40]:
# Establish connection
if connection_manager.establish_connection():
    # Bidirectional communication: Device sends message to Server
    connection_manager.send_from_device(b"Hello from IoT Device")
    
    # Bidirectional communication: Server sends message to Device
    connection_manager.send_from_server(b"Hello from IoT Server")
    
    # Close connection (vault update happens here)
    connection_manager.close_connection()
else:
    print("IoTConnectionManager: Connection failed.")


=== Starting Connection Establishment ===
=== Connection Established Successfully ===

=== Device to Server Communication ===
Device sent encrypted message:  b'e\xc2\xee\x1aMS\x8e\xaf\xcb\xb8`yQ\xf9\xeaH\xdcQ\x97}\x99\xc4\x85\xd9\x95\xe3Q\xf7\xd7\xef\xa8T\t\xd1o\xb3\x08\x9eCV-\x9c\x85)\x8d\x8d\xa6^'
Server decrypted message:  b'Hello from IoT Device'

=== Server to Device Communication ===
Server sent encrypted message:  b'\x06o\xa00\xf0\x98\xf5@\x8a_\xb7\x8eU\xa5\xe2(\xe6\x89\xaa\x07r\xa4Iy\x1b\xd2S6\xd7\xfb\xf1?\r\xfb\x0b\xed\xcc\xc5\xfc\xbeFl\xdd\x1dC&\x1b\x1b'
Device decrypted message:  b'Hello from IoT Server'

=== Closing Connection ===
IoTConnectionManager: Connection closed and vaults updated.



In this code cell, the `send_from_server` method of the `IoTConnectionManager` class is called with the message `"Hello from IoT Server"`. The purpose of this method is to send a message from the server to the device, which involves encrypting the message, transmitting it, and having the device decrypt it. 

However, before attempting to send the message, the code checks if a connection has been established (`self.connection_established`). In this case, since no prior connection has been successfully established (as seen in the previous steps where the `establish_connection` method might not have been called or failed), the connection status is `False`.

In [42]:
connection_manager.send_from_server(b"Hello from IoT Server")

IoTConnectionManager: Cannot send message. No active connection.


In this code cell, the `close_connection` method of the `IoTConnectionManager` class is called. The `close_connection` method is responsible for closing the active connection between the device and the server, and it also updates the vaults of both the device and the server as part of the secure communication protocol.

Before attempting to close the connection, the method checks if the connection has been established by verifying the `self.connection_established` attribute. If the connection was never successfully established (i.e., if the `connection_established` flag is `False`), the method will not proceed with closing the connection. 


In [43]:
connection_manager.close_connection()

IoTConnectionManager: No active connection to close.
